# Objekterkennung IntSys

In diesem Projekt sollen Objekte auf einem Drehteller mithilfe eines Ultraschall Sensors (HCSR04) erkannt werden.

Die gemessenen Daten befinden sich im Unterordner "messdaten"

### Primär orientieren wir uns an der digits-erkennen Übung, und benutzen sklearn

In [32]:
import matplotlib.pyplot as plt
from sklearn import datasets, metrics, svm
from sklearn.model_selection import train_test_split
import pandas as pd
import os

In [45]:
current_directory = os.getcwd()

# Specify the relative path from the current working directory
directory = "annesbl/intsys/messdaten/"
file = "logfile_deo_dose_53mm.txt"
file_path = os.path.join(directory, file)
df = pd.read_csv(file_path, header=None)
#txt und nicht csv file


FileNotFoundError: [Errno 2] No such file or directory: 'annesbl/intsys/messdaten/logfile_deo_dose_53mm.txt'

# Datenformat

Jede log-Datei in den Messdaten besteht aus 10 Umdrehungen.
Zum modellieren, wird die raw data 50/50 geteilt, und anschliessend vorbereitet

Messdateninfos:
Die Messdaten haben jeweils 61 Messpunkte/Umdrehung

In [43]:
directory = "annesbl/intsys/messdaten/"

file = "logfile_deo_dose_53mm.txt"
file_path = os.path.join(directory, file)
df = pd.read_csv(file_path, header=None)
def label_data(df, label):
    df = df.drop(df.columns[0], axis=1)
    df['Label'] = label
    
    """ergänzt dataframe mit einer column, da steht welches objekt gemessen wurde
    """
    return df_labeled

labels = ['hallo']
df = label_data(df,labels)
print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'annesbl/intsys/messdaten/logfile_deo_dose_53mm.txt'

In [44]:
def transform_data(df):
    """
    input 305 measurement points, > 5 turns * 61 measurements
    für jeden messpunkt einer drehung die nächsten 61 punkte anhängen
         > ACHTUNG!: nur wenn es 61 oder mehr messpunkte bis zum ende des input df gibt!
    """

    return df_single_measurements

def label_data(df, label):
    df = df.drop(df.columns[0], axis=1)
    df['Label'] = label
    
    """ergänzt dataframe mit einer column, da steht welches objekt gemessen wurde
    """
    return df_labeled

labels = ['hallo']
df = label_data(df,labels)
print(df)

NameError: name 'df' is not defined